# Performing a Combination

We'll demonstrate how a combination works by combining everything we've learned so far.

## Loading the Workspace

To do so, we'll use a simple workspace to demonstrate functionality of combinations.

In [1]:
import json

spec = json.load(open("data/2-bin_1-channel.json"))

In [2]:
import pyhf

workspace = pyhf.Workspace(spec)

## Combine Workspaces

Let's just try to combine naively right now.

In [3]:
pyhf.Workspace.combine(workspace, workspace)

InvalidWorkspaceOperation: Workspaces cannot have any channels in common with the same name: {'singlechannel'}. You can also try a different join operation: ['none', 'outer', 'left outer', 'right outer'].

As we can see, we can't just combine a workspace with itself if it has some channel names in common. We try very hard in `pyhf` to make sure a combination "makes sense".

Let's go ahead and rename the channel (as well as the measurement). Then try to combine.

In [4]:
other_workspace = workspace.rename(channels={'singlechannel': 'othersinglechannel'},
                                   modifiers={'uncorr_bkguncrt': 'otheruncorr_bkguncrt'},
                                   measurements={'Measurement': 'OtherMeasurement'})

combined_workspace = pyhf.Workspace.combine(workspace, other_workspace)

And did we combine?

In [5]:
print(f'    channels: {combined_workspace.channels}')
print(f'       nbins: {combined_workspace.channel_nbins}')
print(f'     samples: {combined_workspace.samples}')
print(f'   modifiers: {combined_workspace.modifiers}')
print(f'  parameters: {combined_workspace.parameters}')
print(f'measurements: {combined_workspace.measurement_names}')

    channels: ['othersinglechannel', 'singlechannel']
       nbins: {'singlechannel': 2, 'othersinglechannel': 2}
     samples: ['background', 'signal']
   modifiers: [('mu', 'normfactor'), ('otheruncorr_bkguncrt', 'shapesys'), ('uncorr_bkguncrt', 'shapesys')]
  parameters: ['mu', 'otheruncorr_bkguncrt', 'uncorr_bkguncrt']
measurements: ['Measurement', 'OtherMeasurement']


Indeed. And at this point, we can just use all the same functionality we expect of pyhf, such as performing a fit:

In [6]:
model = workspace.model()
data = workspace.data(model)

pyhf.infer.hypotest(1.0, data, model, qtilde=True)

array(0.48563995)

In [7]:
other_model = other_workspace.model()
other_data = other_workspace.data(other_model)

pyhf.infer.hypotest(1.0, other_data, other_model, qtilde=True)

array(0.48563995)

In [8]:
combined_model = combined_workspace.model()
combined_data = combined_workspace.data(combined_model)

pyhf.infer.hypotest(1.0, combined_data, combined_model, qtilde=True)

multiple measurements defined. Taking the first measurement.


array(0.35661112)